In [1]:
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 
import datetime

In [2]:
today=str(datetime.datetime.today().strftime('%d_%m_%Y'))

In [3]:
INC = pd.read_excel(r'C:\NotBackedUp\SOE\source_files\CHG\change_request.xlsx')

In [4]:
list(INC)

['Number',
 'Active',
 'Activity due',
 'Additional assignee list',
 'Approval',
 'Approval history',
 'Approval set',
 'Assigned to',
 'Change Owner group',
 'Business duration',
 'Incident Number',
 'Created',
 'State',
 'Name',
 'Close code']

In [5]:
INC = INC[INC['State'] == 'Closed']

In [6]:
INC = INC[['Number','Name','Created','State','Close code']]

In [7]:
INC['Name'] = INC['Name'].str.strip('[PROD]')
INC['Name'] = INC['Name'].str.strip()

In [8]:
INC = INC.rename(columns={'Name':'business_service_name','Close code':'CloseCode'})

In [9]:
model1map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\model1_mapping.csv')
siam_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\SIAM_ServiceMapping.csv')

In [10]:
siam_map = siam_map.loc[siam_map.notnull().sum(1).groupby(siam_map.Name).idxmax()]
model1map = model1map.loc[model1map.notnull().sum(1).groupby(model1map.Name).idxmax()]

In [11]:
CHG_Implementation_Model1 = INC.merge(model1map,how='left',left_on=['business_service_name'],right_on=['Name'])
CHG_Implementation_SIAM= INC.merge(siam_map,how='left',left_on=['business_service_name'],right_on=['Name'])

In [12]:
CHG_Implementation_Model1 = CHG_Implementation_Model1[['Number','business_service_name','Created','State','CloseCode','Name','Domain','Tribe_TechArea']]

In [13]:
CHG_Implementation_SIAM = CHG_Implementation_SIAM[['Number','business_service_name','Created','State','CloseCode','Name','Domain','Tribe_TechArea']]

In [14]:
CHG_Implementation_Model1['Name'] = CHG_Implementation_Model1['Name'].fillna(0)

In [15]:
CHG_Implementation_SIAM['Name'] = CHG_Implementation_SIAM['Name'].fillna(0)

In [16]:
CHG_Implementation_Model1 = CHG_Implementation_Model1[CHG_Implementation_Model1['Name'] != 0]

In [17]:
CHG_Implementation_SIAM = CHG_Implementation_SIAM[CHG_Implementation_SIAM['Name'] != 0]

In [18]:
CHG_Implementation_SIAM.CloseCode.unique()

array(['Successful', 'Rolled back', 'Implemented with incidents',
       'Partially implemented but business objective met',
       'Partially implemented', 'Unsuccessful – Objective not met'], dtype=object)

In [19]:
CHG_Model1 = CHG_Implementation_Model1.groupby(['business_service_name','Domain','Tribe_TechArea','CloseCode'])['Number'].count().reset_index(name='CHG_Implemented')

In [20]:
CHG_SIAM = CHG_Implementation_SIAM.groupby(['business_service_name','Domain','Tribe_TechArea','CloseCode'])['Number'].count().reset_index(name='CHG_Implemented')

In [21]:
CHG_Implementation_Model1.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\RAW_CHG_IMPLEMENTED_MODEL1_'+today+'.csv',index=None)
CHG_Implementation_SIAM.to_csv(r'C:\NotBackedUp\SOE\SIAM\CSV\RAW_CHG_IMPLEMENTED_SIAM_'+today+'.csv',index=None)

In [22]:
CHG_SIAM.to_csv(r'C:\NotBackedUp\SOE\SIAM\CSV\CHG_IMPLEMENTED_SIAM_'+today+'.csv',index=None)
CHG_Model1.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\CHG_IMPLEMENTED_MODEL1_'+today+'.csv',index=None)